# ShuffleNetV2_0.5 pth fix

In [1]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [2]:
import torch
from torch import nn as nn
import torchvision
from src.utils.common import get_label_counts, read_yaml
from src.model import Model

In [3]:
class Shufflenet_v205(nn.Module):
    def __init__(self, num_classes:int = 9):
        super().__init__()
        self.model = torchvision.models.shufflenet_v2_x0_5(pretrained=True)
        self.model.fc = torch.nn.Linear(1024, num_classes)

    def forward(self, x):
        return self.model(x)

In [4]:
# load pretrained
ShModel = Shufflenet_v205()

In [5]:
# load model used
model_config = read_yaml(cfg="configs/model/shufflenetv2.yaml")
model_used = Model(cfg=model_config)

In [6]:
print(len(ShModel.state_dict()), len(model_used.state_dict()))

338 338


In [7]:
pretrained_state_dict = ShModel.state_dict()
keys_load = [x for x in pretrained_state_dict.keys()]
keys_load = {x:y for x,y in zip(keys_load, model_used.state_dict().keys())}
for before, after in keys_load.items():
    pretrained_state_dict[after] = pretrained_state_dict.pop(before)
model_used.load_state_dict(pretrained_state_dict)
print("load_state_dict completed.")

load_state_dict completed.


In [8]:
torch.save(pretrained_state_dict, 'fixed_pt.pt')